# Using VGG16 to solve Kaggle's Dogs vs Cats problem

### Structure folders

In [1]:
%cd data/test

/home/ubuntu/work/kaggle/dogs-and-cats/data/test


In [2]:
%mkdir unknown

In [3]:
!mv *.jpg unknown/

In [4]:
%cd ../train

/home/ubuntu/work/kaggle/dogs-and-cats/data/train


In [5]:
%mkdir cats
%mkdir dogs
!mv cat.* cats
!mv dog.* dogs

In [6]:
%cd ../
%mkdir valid
%mkdir valid/dogs
%mkdir valid/cats

/home/ubuntu/work/kaggle/dogs-and-cats/data


In [7]:
%%bash
N=1000;
for i in ./train/dogs/*; do
  [ $((N--)) = 0 ] && break
  cp -t "./valid/dogs" -- "$i"
done

In [8]:
%%bash
N=1000;
for i in ./train/cats/*; do
  [ $((N--)) = 0 ] && break
  cp -t "./valid/cats" -- "$i"
done

## Model

In [1]:
# path = ""
path = "data/"

A few basic libraries that we'll need for the initial exercises:

In [2]:
from __future__ import division,print_function

import os, json, csv
from glob import glob
import numpy as np
np.set_printoptions(precision=4, linewidth=100)
from matplotlib import pyplot as plt

# Use a pretrained VGG model with our **Vgg16** class

Our first step is simply to use a model that has been fully created for us, which can recognise a wide variety (1,000 categories) of images. We will use 'VGG', which won the 2014 Imagenet competition, and is a very simple model to create and understand. The VGG Imagenet team created both a larger, slower, slightly more accurate model (*VGG  19*) and a smaller, faster model (*VGG 16*). We will be using VGG 16 since the much slower performance of VGG19 is generally not worth the very minor improvement in accuracy.

We have created a python class, *Vgg16*, which makes using the VGG 16 model very straightforward. 

## The punchline: state of the art custom model in 7 lines of code

Here's everything you need to do to get >97% accuracy on the Dogs vs Cats dataset - we won't analyze how it works behind the scenes yet, since at this stage we're just going to focus on the minimum necessary to actually do useful work.

In [3]:
# As large as you can, but no larger than 64 is recommended. 
# If you have an older or cheaper GPU, you'll run out of memory, so will have to decrease this.
batch_size=64

In [4]:
# Import our class, and instantiate
from vgg16 import Vgg16

Using Theano backend.
Using gpu device 0: GeForce GTX 970 (CNMeM is disabled, cuDNN 5103)
/home/nima/anaconda3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)


## Use Vgg16 for basic image recognition

In [47]:
vgg = Vgg16()

In [48]:
batches = vgg.get_batches(path+'train', batch_size=batch_size)
val_batches = vgg.get_batches(path+'valid', batch_size=batch_size)
test_batches = vgg.get_batches(path+'test', batch_size=batch_size, shuffle=False)
file_ids = list(map(lambda x: int(x.split("/")[1].replace(".jpg", "")), test_batches.filenames))

Found 25000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Found 12500 images belonging to 1 classes.


In [49]:
vgg.finetune(batches)

In [50]:
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
25000/25000 [==============================] - 500s - loss: 2.2321 - acc: 0.8598 - val_loss: 0.6495 - val_acc: 0.9580


In [51]:
vgg.model.save_weights(path+'results/ft2.h5')

In [45]:
vgg.model.load_weights(path+'results/ft2.h5')

In [52]:
vgg.model.optimizer.lr = 0.01
vgg.fit(batches, val_batches, nb_epoch=1)

Epoch 1/1
25000/25000 [==============================] - 499s - loss: 0.4847 - acc: 0.9693 - val_loss: 0.3356 - val_acc: 0.9790


In [54]:
vgg.model.save_weights(path+'results/ft3.h5')

In [55]:
predictions = vgg.model.predict_generator(test_batches, len(test_batches.filenames), max_q_size=batch_size*2)

In [56]:
csvfile = open('submission.csv', 'w')
writer = csv.writer(csvfile, delimiter=",")

preds = predictions[:]
preds = np.delete(preds, 0, axis=1)
preds = np.clip(preds, 0.05, 0.96)
data = np.column_stack((file_ids, preds.flatten()))

np.savetxt("submission.csv", data, fmt=['%d', '%.18f'], delimiter=",", header="id,label")

In [57]:
from IPython.lib.display import FileLink
FileLink("submission.csv")

/home/nima/workspace/kaggle/dogs-and-cats/submission.csv

In [ ]:
csvfile = open('submission.csv', 'w')
writer = csv.writer(csvfile, delimiter=",")
index = 0
batch_count = 0
results = []
print("total length: %d" % (len(file_ids)))

batch,labels = next(test_batches, None)
while(batch is not None and len(batch) > 0 and index < len(file_ids)):
    preds, idxs, labels = vgg.predict(batch, False)
    for img_class in idxs:
        writer.writerow([file_ids[index], img_class])b
        index = index + 1
    batch_count = batch_count + 1
    batch,labels = next(test_batches, None)

print("total batches: %s" % (batch_count, index))
csvfile.close()